# Creating flows from your laptop machine learning code

To introduce data scientists to [Metaflow](https://docs.metaflow.org/), it will be key to show them how they can take their pre-existing ML code and turn it into flows. Arguably, the 3 most practical types of models are

* Random forests
* Boosted trees, and
* Neural nets.

To this end, in what follows, we show how you would take code for each of these types of models and turn it into a Metaflow.

## Setup instructions

_Note:_ Include setup instructions in README. Also perhaps include them here, depending on whether you include command line stuff later.

We'll be using `conda` to install the necessary packages but you can also use `pip` or `virtualenv`. To use `conda`, install the Anaconda distribution from [here](https://www.anaconda.com/products/individual).
Using the command line, execute

```bash
conda env create -f env.yml
```
to create your environment. You can then activate it by executing

```bash
conda activate full-stack-metaflow
```

## Your Laptop Machine Learning Refresher

### Random Forests

This is typical random forest code:

In [1]:
#Import scikit-learn dataset library
from sklearn import datasets

#Load dataset
iris = datasets.load_iris()
X = iris.data
y = iris.target
X.shape, y.shape

((150, 4), (150,))

In [2]:
# https://scikit-learn.org/stable/modules/ensemble.html#forest
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(max_depth=None, min_samples_split=2,
    random_state=0)
scores = cross_val_score(clf, X, y, cv=5)
print(scores)


clf = RandomForestClassifier(n_estimators=10, max_depth=None,
    min_samples_split=2, random_state=0)
scores = cross_val_score(clf, X, y, cv=5)
print(scores)


clf = ExtraTreesClassifier(n_estimators=10, max_depth=None,
    min_samples_split=2, random_state=0)
scores = cross_val_score(clf, X, y, cv=5)
print(scores)

[0.96666667 0.96666667 0.9        0.96666667 1.        ]
[0.96666667 0.96666667 0.9        0.93333333 1.        ]
[0.96666667 0.96666667 0.93333333 0.9        1.        ]


### Boosted trees



This is typical boosted tree code:

In [3]:
import xgboost as xgb
# read in data
dtrain = xgb.DMatrix('../data/agaricus.txt.train')
dtest = xgb.DMatrix('../data/agaricus.txt.test')
# specify parameters via map|
param = {'max_depth':2, 'eta':1, 'objective':'binary:logistic' }
num_round = 2
bst = xgb.train(param, dtrain, num_round)
# make prediction
preds = bst.predict(dtest)
print(preds)

[18:14:01] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.28583017 0.9239239  0.28583017 ... 0.9239239  0.05169873 0.9239239 ]


### Neural nets

This is (somewhat) typical deep learning code (we're using Keras & TensorFlow but you can use PyTorch, fast.ai, JAX, and/or PyTorch Lightning, among others <-- check if all of this is true, HBA):

In [4]:
# https://keras.io/examples/vision/mnist_convnet/
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [5]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [6]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                1

2022-03-16 18:14:11.542968: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

2022-03-16 18:14:14.544811: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/15
422/422 [==============================] - 12s 27ms/step - loss: 0.7205 - accuracy: 0.7823 - val_loss: 0.0841 - val_accuracy: 0.9775
Epoch 2/15
422/422 [==============================] - 11s 26ms/step - loss: 0.1300 - accuracy: 0.9619 - val_loss: 0.0623 - val_accuracy: 0.9825
Epoch 3/15
422/422 [==============================] - 11s 26ms/step - loss: 0.0885 - accuracy: 0.9725 - val_loss: 0.0460 - val_accuracy: 0.9868
Epoch 4/15
422/422 [==============================] - 11s 26ms/step - loss: 0.0757 - accuracy: 0.9757 - val_loss: 0.0436 - val_accuracy: 0.9877
Epoch 5/15
422/422 [==============================] - 11s 27ms/step - loss: 0.0652 - accuracy: 0.9794 - val_loss: 0.0384 - val_accuracy: 0.9888
Epoch 6/15
422/422 [==============================] - 11s 26ms/step - loss: 0.0540 - accuracy: 0.9838 - val_loss: 0.0354 - val_accuracy: 0.9905
Epoch 7/15
422/422 [==============================] - 11s 26ms/step - loss: 0.0511 - accuracy: 0.9841 - val_loss: 0.0325 - val_accuracy:

In [8]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])


Test loss: 0.026157839223742485
Test accuracy: 0.9912999868392944


In the next lesson, we'll take these machine learning scripts and turn them into (meta)flows.